In [1]:
# !pip install PytorchCML

In [1]:
import sys
sys.path.append("../../src/")

from itertools import product

from PytorchCML import losses, models, samplers, regularizers, evaluators, trainers
import torch
from torch import nn, optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [3]:
movielens = pd.read_csv(
  'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
  sep='\t', header=None, index_col=None,
)
movielens.columns = ["user_id", "item_id", "rating", "timestamp"]
movielens.user_id -= 1
movielens.item_id -= 1
movielens.rating = (movielens >= 4).astype(int)
n_user = movielens.user_id.nunique()
n_item = movielens.item_id.nunique()

train, test = train_test_split(movielens)


# all user item pairs
df_all = pd.DataFrame(
    [[u, i] for u,i in product(range(n_user), range(n_item))],
    columns=["user_id", "item_id"]
)

# frag train pairs
df_all = pd.merge(
    df_all, 
    train[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
)

# remove train pairs
test = pd.merge(
    df_all[df_all.rating.isna()][["user_id", "item_id"]], 
    test[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
).fillna(0)

# numpy array
train_set = train[train.rating == 1][["user_id", "item_id"]].values
test_set = test[["user_id", "item_id", "rating"]].values

# to torch.Tensor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)


## Defalt

In [3]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)


In [4]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:18<00:00, 49.84it/s]


In [5]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.018667,0.033051,0.002104,0.019511,0.040509,0.003674,0,NaN
0,0.079036,0.119477,0.006106,0.078784,0.132808,0.009886,10,0.520977
0,0.240874,0.331478,0.028101,0.244704,0.348947,0.048870,20,0.366803


## Strict Negative

In [3]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=True)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)


In [4]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:15<00:00, 59.91it/s]


In [5]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.015895,0.028455,0.001643,0.015187,0.033086,0.002452,0,NaN
0,0.057471,0.091110,0.004146,0.063293,0.107319,0.008038,10,1212.098991
0,0.254217,0.345263,0.029643,0.251258,0.355607,0.048304,20,738.998521


## Global Orthogonal Regularization

In [6]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
regs = [regularizers.GlobalOrthogonalRegularizer(weight=1e-2)]
criterion = losses.SumTripletLoss(margin=1, regularizers=regs).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=True)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)

In [7]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:16<00:00, 57.42it/s]


In [8]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.014230,0.025097,0.001684,0.014524,0.030877,0.002610,0,NaN
0,0.074575,0.116472,0.005911,0.080112,0.130880,0.010192,10,0.447714
0,0.274891,0.366826,0.033157,0.272943,0.381603,0.055453,20,0.278262


## Two Stage

In [32]:
item_count = train.groupby("item_id")["user_id"].count()
count_index = np.array(item_count.index)
neg_weight = np.zeros(n_item)
neg_weight[count_index] = item_count ** 0.1

In [36]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

regs = [regularizers.GlobalOrthogonalRegularizer(weight=1e-3)]
criterion = losses.MinTripletLoss(margin=1, regularizers=regs).to(device)
sampler = samplers.TwoStageSampler(
    train_set, n_user, n_item, 
    neg_weight=neg_weight, n_neg_samples=5,
    device=device, strict_negative=False
)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)

In [37]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:16<00:00, 58.30it/s]


In [38]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.017962,0.034023,0.001885,0.017329,0.040703,0.003181,0,NaN
0,0.225361,0.311594,0.023387,0.219194,0.322207,0.038719,10,1.044481
0,0.378223,0.525097,0.057584,0.339713,0.521593,0.078961,20,1.006334


## model weighted negative sampler

In [4]:
def svd_init(X, dim):
    """
    Args :
        X : csr_matrix which element is 0 or 1.
        dim : number of dimention
    """
    svd = TruncatedSVD(n_components=10)
    U_ = svd.fit_transform(X)
    V_ = svd.components_

    s = (U_.sum(axis=1).mean() + V_.sum(axis=0).mean()) / 2
    U = 2 ** 0.5 * U_ - (1 / n_dim) ** 0.5 * s * np.ones_like(U_)
    V = 2 ** 0.5 * V_ + (1 / n_dim) ** 0.5 / s * np.ones_like(V_)
    ub = -(2 / n_dim) ** 0.5 * U_.sum(axis=1) / s
    vb = (2 / n_dim) ** 0.5 * V_.sum(axis=0) * s

    return U, V, ub, vb

In [5]:
n_dim = 10
X = csr_matrix(
    (np.ones(train_set.shape[0]), (train_set[:,0], train_set[:,1])),
    shape=[n_user, n_item]
)
U, V, ub, vb = svd_init(X, n_dim)
neg_weight_model = models.LogitMatrixFactorization(
    n_user, n_item, n_dim, max_norm=None,
    user_embedding_init = torch.Tensor(U), 
    item_embedding_init = torch.Tensor(V.T),
    user_bias_init = torch.Tensor(ub), 
    item_bias_init = torch.Tensor(vb)
).to(device)
neg_weight_model.link_weight = lambda x : 1 - torch.sigmoid(x)

In [6]:
lr = 1e-3
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(
    train_set, n_user, n_item, 
    neg_weight=neg_weight_model,
    device=device, strict_negative=False
)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.CMLTrainer(model, optimizer, criterion, sampler)

In [7]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:20<00:00, 45.11it/s]


In [8]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.016209,0.030399,0.001955,0.01690,0.038264,0.003477,0,NaN
0,0.051292,0.078208,0.004285,0.05536,0.094836,0.007510,10,0.572125
0,0.233268,0.322552,0.030232,0.23401,0.336276,0.049536,20,0.430135
